# Geowave GPX Demo
This Demo runs KMeans on the GPX dataset consisting of approximately 285 million point locations. We use a cql filter to reduce the KMeans set to a bounding box over Berlin, Germany. Simply focus a cell and use [SHIFT + ENTER] to run the code.

# Import pixiedust
Start by importing pixiedust which if all bootstrap and install steps were run correctly.
You should see below for opening the pixiedust database successfully with no errors.
Depending on the version of pixiedust that gets installed, it may ask you to update.
If so, run this first cell.

In [ ]:
#!pip install --user --upgrade pixiedust

In [1]:
import pixiedust

Pixiedust database opened successfully


Pixiedust also allows us to monitor spark job progress directly from the notebook. Simply run the cell below and anytime a spark job is run from the notebook you should see incremental progress shown in the output below.
*NOTE* If this function fails or produces a error often this is just a link issue between pixiedust and python the first time pixiedust is imported. Restart the Kernel and rerun the cells to fix the error.

In [17]:
pixiedust.enableJobMonitor()

▸,:,


Spark Job Progress Monitor already enabled


# Creating the SQLContext and inspecting pyspark Context
Pixiedust imports pyspark and the SparkContext + SparkSession should be already available through the "sc" and "spark" variables respectively.

In [3]:
# Print Spark info and create sql_context
print('Spark Version: {0}'.format(sc.version))
print('Python Version: {0}'.format(sc.pythonVer))
print('Application Name: {0}'.format(sc.appName))
print('Application ID: {0}'.format(sc.applicationId))
print('Spark Master: {0}'.format( sc.master))

sql_context = SQLContext(sc, sparkSession=spark)

Spark Version: 2.2.0
Python Version: 3.5
Application Name: pyspark-shell
Application ID: application_1509971717307_0002
Spark Master: yarn


# Download and ingest the GPX data
*NOTE* Depending on cluster size sometimes the copy can fail. This appears to be a race condition error with the copy command when downloading the files from s3. This may make the following import into acccumulo command fail. You can check the accumulo tables by looking at port 9995 of the emr cluster. There should be 5 tables after importing.

In [4]:
%%bash
s3-dist-cp -D mapreduce.task.timeout=60000000 --src=s3://geowave-gpx-data/gpx --dest=hdfs://$HOSTNAME:8020/tmp/ 

▸,:,


17/11/06 12:43:06 INFO s3distcp.S3DistCp: Running with args: -libjars /usr/share/aws/emr/s3-dist-cp/lib/guava-15.0.jar,/usr/share/aws/emr/s3-dist-cp/lib/s3-dist-cp-2.6.0.jar,/usr/share/aws/emr/s3-dist-cp/lib/s3-dist-cp.jar -D mapreduce.task.timeout=60000000 --src=s3://geowave-gpx-data/gpx --dest=hdfs://ip-10-0-0-241:8020/tmp/ 
17/11/06 12:43:07 INFO s3distcp.S3DistCp: S3DistCp args: --src=s3://geowave-gpx-data/gpx --dest=hdfs://ip-10-0-0-241:8020/tmp/ 
17/11/06 12:43:07 INFO s3distcp.S3DistCp: Using output path 'hdfs:/tmp/ee7e2272-fd20-4081-a4db-e76ff7fb8500/output'
17/11/06 12:43:07 INFO s3distcp.S3DistCp: GET http://169.254.169.254/latest/meta-data/placement/availability-zone result: us-east-1f
17/11/06 12:43:09 INFO s3distcp.FileInfoListing: Opening new file: hdfs:/tmp/ee7e2272-fd20-4081-a4db-e76ff7fb8500/files/1
17/11/06 12:43:09 INFO s3distcp.S3DistCp: Created 1 files to copy 66 files 
17/11/06 12:43:09 INFO s3distcp.S3DistCp: Reducer number: 63
17/11/06 12:43:10 INFO impl.Timelin

In [5]:
%%bash
/opt/accumulo/bin/accumulo shell -u root -p secret -e "importtable geowave.germany_gpx_SPATIAL_IDX /tmp/spatial"
/opt/accumulo/bin/accumulo shell -u root -p secret -e "importtable geowave.germany_gpx_GEOWAVE_METADATA /tmp/metadata"

▸,:,


2017-11-06 12:45:51,211 [conf.ConfigSanityCheck] WARN : Use of instance.dfs.uri and instance.dfs.dir are deprecated. Consider using instance.volumes instead.
2017-11-06 12:45:51,926 [htrace.SpanReceiverBuilder] ERROR: SpanReceiverBuilder cannot find SpanReceiver class org.apache.accumulo.tracer.ZooTraceClient: disabling span receiver.
2017-11-06 12:45:51,926 [trace.DistributedTrace] WARN : Failed to load SpanReceiver org.apache.accumulo.tracer.ZooTraceClient
2017-11-06 12:45:54,879 [conf.ConfigSanityCheck] WARN : Use of instance.dfs.uri and instance.dfs.dir are deprecated. Consider using instance.volumes instead.
2017-11-06 12:45:55,601 [htrace.SpanReceiverBuilder] ERROR: SpanReceiverBuilder cannot find SpanReceiver class org.apache.accumulo.tracer.ZooTraceClient: disabling span receiver.
2017-11-06 12:45:55,602 [trace.DistributedTrace] WARN : Failed to load SpanReceiver org.apache.accumulo.tracer.ZooTraceClient
2017-11-06 12:45:55,921 [impl.TableOperationsImpl] INFO : Imported table s

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/accumulo-1.8.1/lib/slf4j-log4j12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/accumulo-1.8.1/lib/slf4j-log4j12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


# Setup Datastores

In [6]:
%%bash
# clear out potential old runs
geowave config rmstore kmeans_gpx
geowave config rmstore germany_gpx_accumulo

# configure geowave connection params for name stores "germany_gpx_accumulo" and "kmeans_hbase"
geowave config addstore germany_gpx_accumulo --gwNamespace geowave.germany_gpx -t accumulo --zookeeper $HOSTNAME:2181 --instance accumulo --user root --password secret
geowave config addstore kmeans_gpx --gwNamespace geowave.kmeans -t hbase --zookeeper $HOSTNAME:2181

▸,:,


# Run KMeans
Run Kmeans on the reduced dataset over Berlin, Germany. Once the spark job begins running you should be able to monitor its progress from the cell with pixiedust, or you can monitor the progress from the spark history server on the emr cluster.

In [7]:
%%bash

geowave remote clear kmeans_gpx

▸,:,


06 Nov 12:48:50 INFO [zookeeper.RecoverableZooKeeper] - Process identifier=hconnection-0x39b43d60 connecting to ZooKeeper ensemble=ip-10-0-0-241:2181
06 Nov 12:48:51 INFO [client.HBaseAdmin] - Started disable of geowave.kmeans_GEOWAVE_METADATA
06 Nov 12:48:53 INFO [client.HBaseAdmin] - Disabled geowave.kmeans_GEOWAVE_METADATA
06 Nov 12:49:01 INFO [client.HBaseAdmin] - Deleted geowave.kmeans_GEOWAVE_METADATA
06 Nov 12:49:01 INFO [client.HBaseAdmin] - Started disable of geowave.kmeans_SPATIAL_IDX
06 Nov 12:49:05 INFO [client.HBaseAdmin] - Disabled geowave.kmeans_SPATIAL_IDX
06 Nov 12:50:14 INFO [client.HBaseAdmin] - Deleted geowave.kmeans_SPATIAL_IDX


In [8]:
#grab classes from jvm
hbase_options_class = sc._jvm.mil.nga.giat.geowave.datastore.hbase.operations.config.HBaseRequiredOptions
accumulo_options_class = sc._jvm.mil.nga.giat.geowave.datastore.accumulo.operations.config.AccumuloRequiredOptions
kmeans_runner_class = sc._jvm.mil.nga.giat.geowave.analytic.javaspark.kmeans.KMeansRunner
query_options_class = sc._jvm.mil.nga.giat.geowave.core.store.query.QueryOptions
geowave_rdd_class = sc._jvm.mil.nga.giat.geowave.analytic.javaspark.GeoWaveRDD
sf_df_class = sc._jvm.mil.nga.giat.geowave.analytic.javaspark.sparksql.SimpleFeatureDataFrame
byte_array_class = sc._jvm.mil.nga.giat.geowave.core.index.ByteArrayId

▸,:,


In [9]:
#setup input datastore
input_store = accumulo_options_class()
input_store.setInstance('accumulo')
input_store.setUser('root')
input_store.setPassword('secret')
input_store.setZookeeper(os.environ['HOSTNAME'] + ':2181')
input_store.setGeowaveNamespace('geowave.germany_gpx')

#Setup output datastore
output_store = hbase_options_class()
output_store.setZookeeper(os.environ['HOSTNAME'] + ':2181')
output_store.setGeowaveNamespace('geowave.kmeans')

#Create a instance of the runner
kmeans_runner = kmeans_runner_class()

input_store_plugin = input_store.createPluginOptions()
output_store_plugin = output_store.createPluginOptions()

▸,:,


In [10]:
#set the appropriate properties
#We want it to execute using the existing JavaSparkContext wrapped by python.
kmeans_runner.setJavaSparkContext(sc._jsc)

kmeans_runner.setAdapterId('gpxpoint')
kmeans_runner.setNumClusters(8)
kmeans_runner.setInputDataStore(input_store_plugin)
kmeans_runner.setOutputDataStore(output_store_plugin)
kmeans_runner.setCqlFilter("BBOX(geometry,  13.3, 52.45, 13.5, 52.5)")
kmeans_runner.setCentroidTypeName('mycentroids')
kmeans_runner.setHullTypeName('myhulls')
kmeans_runner.setGenerateHulls(True)
kmeans_runner.setComputeHullData(True)
#execute the kmeans runner
kmeans_runner.run()

▸,:,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Load Centroids into DataFrame and display

In [11]:
# Create the dataframe and get a rdd for the output of kmeans
sf_df = sf_df_class(spark._jsparkSession)
adapter_id = byte_array_class('mycentroids')

queryOptions = None
adapterIt = output_store_plugin.createAdapterStore().getAdapters()
adapterForQuery = None
while (adapterIt.hasNext()):
    adapter = adapterIt.next()
    if (adapter.getAdapterId().equals(adapter_id)):
        adapterForQuery = adapter
        queryOptions = query_options_class(adapterForQuery)
        break

output_rdd = geowave_rdd_class.rddForSimpleFeatures(sc._jsc.sc(), output_store_plugin, None, queryOptions)

sf_df.init(output_store_plugin, adapter_id)

df = sf_df.getDataFrame(output_rdd)
# Convert Java DataFrame to Python DataFrame
import pyspark.mllib.common as convert
py_df = convert._java2py(sc, df)

py_df.createOrReplaceTempView('mycentroids')

df = sql_context.sql("select * from mycentroids")

display(df)

geom,ClusterIndex
POINT (13.317627329946358 52.47922516681432),3
POINT (13.347765974127926 52.4746733116948),2
POINT (13.483286182032337 52.47720923883304),1
POINT (13.45083572257175 52.4938577939659),5
POINT (13.367694913503392 52.489673493155),0
POINT (13.388505938718911 52.47600577751235),7
POINT (13.415824769373659 52.488445643872026),4
POINT (13.443916096451614 52.46347539442979),6


# Parse DataFrame data into lat/lon columns and display centroids on map
Using pixiedust's built in map visualization we can display data on a map assuming it has the following properties.
- Keys: put your latitude and longitude fields here. They must be floating values. These fields must be named latitude, lat or y and longitude, lon or x.
- Values: the field you want to use to thematically color the map. Only one field can be used.

Also you will need a access token from whichever map renderer you choose to use with pixiedust (mapbox, google).
Follow the instructions in the token help on how to create and use the access token.

In [12]:
# Convert the string point information into lat long columns and create a new dataframe for those.
import pyspark
def parseRow(row):
    lat_start = row.geom.rfind(' ') + 1
    lat_end = row.geom.rfind(')')
    lat = row.geom[lat_start:lat_end]
    lon_start = row.geom.find('(') + 1
    lon_end = row.geom.rfind(' ', lon_start)
    lon = row.geom[lon_start:lon_end]
    return pyspark.sql.Row(lat=float(lat), lon=float(lon), ClusterIndex=row.ClusterIndex)
    
row_rdd = df.rdd
new_rdd = row_rdd.map(lambda row: parseRow(row))
new_df =new_rdd.toDF() 
display(new_df)

# Export KMeans Hulls to DataFrame
If you have some more complex data to visualize pixiedust may not be the best option.

The Kmeans hull generation outputs polygons that would be difficult for pixiedust to display without
creating a special plugin. 

Instead, we can use another map renderer to visualize our data. For the Kmeans hulls we will use ipyleaflet to visualize the data. We will start by grabbing the results for the hull generation and putting them into a DataFrame

In [13]:
# Create the dataframe and get a rdd for the output of kmeans
sf_df_hulls = sf_df_class(spark._jsparkSession)
adapter_id = byte_array_class('myhulls')

queryOptions = None
adapterIt = output_store_plugin.createAdapterStore().getAdapters()
adapterForQuery = None
while (adapterIt.hasNext()):
    adapter = adapterIt.next()
    if (adapter.getAdapterId().equals(adapter_id)):
        adapterForQuery = adapter
        queryOptions = query_options_class(adapterForQuery)
        break

output_rdd_hulls = geowave_rdd_class.rddForSimpleFeatures(sc._jsc.sc(), output_store_plugin, None, queryOptions)

sf_df_hulls.init(output_store_plugin, adapter_id)

df_hulls = sf_df_hulls.getDataFrame(output_rdd_hulls)
# Convert Java DataFrame to Python DataFrame
import pyspark.mllib.common as convert
py_df_hulls = convert._java2py(sc, df_hulls)

py_df_hulls.createOrReplaceTempView('myhulls')

df_hulls = sql_context.sql("select * from myhulls order by Density")

display(df_hulls)

geom,ClusterIndex,Count,Area,Density
"POLYGON ((13.4439443 52.4500005, 13.435513 52.450011, 13.4206216 52.4501, 13.4124933 52.451955, 13.412206 52.452025, 13.4134 52.4556, 13.414353 52.458444, 13.416086 52.460444, 13.41833 52.462976, 13.421366 52.466393, 13.4348 52.4815, 13.4362 52.4812, 13.4450555 52.4791942, 13.451483 52.477729, 13.459015 52.475988, 13.46339 52.470942, 13.463752 52.469908, 13.464018 52.469147, 13.469976 52.452055, 13.468309 52.450024, 13.46598 52.450009, 13.4646785 52.4500018, 13.4644143 52.4500008, 13.4439443 52.4500005))",6,34677,9.702393252889806,3574.0666345050117
"POLYGON ((13.492451 52.450008, 13.49181 52.45005, 13.4700016 52.4520566, 13.4676098 52.4588682, 13.46515 52.46591, 13.4637665 52.4698877, 13.463431 52.470871, 13.4648923 52.4813003, 13.469224 52.489748, 13.470204 52.491657, 13.473088 52.497273, 13.4747891 52.4999943, 13.478891 52.499998, 13.491197 52.4999994, 13.496814 52.499983, 13.496944 52.4999743, 13.497525 52.4998866, 13.4982016 52.4997, 13.4985116 52.4996116, 13.4987416 52.4995333, 13.4999883 52.4990983, 13.499999 52.498989, 13.499999 52.462982, 13.499998 52.452558, 13.499973 52.451963, 13.499892 52.450439, 13.499132 52.450014, 13.497535 52.450012, 13.492451 52.450008))",1,53307,12.117579210859516,4399.1459904984495
"POLYGON ((13.3958883 52.45001, 13.385129 52.4500142, 13.378801 52.45002, 13.3721351 52.4501419, 13.368953 52.450731, 13.36895 52.4507383, 13.368639 52.460234, 13.368483 52.467539, 13.36854 52.46792, 13.36896 52.468902, 13.36923 52.46933, 13.3703966 52.4711074, 13.372795 52.47476, 13.388901 52.4992849, 13.389 52.4994, 13.389111 52.499513, 13.3915598 52.4998056, 13.3931416 52.4995899, 13.3944499 52.49875, 13.394779 52.498349, 13.39517 52.49756, 13.3953339 52.497216, 13.401532 52.483614, 13.410421 52.464095, 13.41233 52.459899, 13.412791 52.457626, 13.413 52.4556, 13.41208 52.452048, 13.405199 52.450043, 13.3958883 52.45001))",7,58564,11.186682643629085,5235.153428916902
"POLYGON ((13.413604 52.457997, 13.413302 52.458018, 13.412699 52.459206, 13.412442 52.459736, 13.410375 52.464214, 13.409933 52.465178, 13.4014989 52.4836902, 13.401017 52.484748, 13.399182 52.488778, 13.395927 52.495927, 13.395442 52.496996, 13.3951663 52.497611, 13.3948768 52.4982626, 13.3945946 52.4996212, 13.3949836 52.4999329, 13.3949967 52.4999417, 13.39697 52.4999667, 13.40787 52.4999899, 13.4156276 52.4999992, 13.41718 52.5, 13.43157 52.5, 13.431861 52.499982, 13.43195 52.49995, 13.43205 52.499428, 13.4328 52.49458, 13.4340306 52.4866159, 13.434322 52.484665, 13.434575 52.48276, 13.434686 52.4813836, 13.420205 52.4650883, 13.416164 52.460547, 13.414092 52.458279, 13.413604 52.457997))",4,42320,7.825824925505086,5407.7366159924195
"POLYGON ((13.313259 52.450003, 13.3032116 52.4500055, 13.3001166 52.4523172, 13.300065 52.4523733, 13.300004 52.454443, 13.300002 52.457984, 13.3000007 52.4686987, 13.3 52.488, 13.3 52.4882, 13.3000035 52.4945487, 13.3000212 52.4986631, 13.30004 52.49872, 13.300058 52.498767, 13.300595 52.499998, 13.307398 52.5, 13.33576 52.5, 13.3360302 52.4999861, 13.336147 52.499844, 13.3358933 52.4981183, 13.33295 52.478627, 13.332758 52.477356, 13.33072 52.46388, 13.329224 52.453997, 13.3291833 52.453845, 13.323001 52.450059, 13.3139971 52.4500033, 13.313259 52.450003))",3,101760,12.123376875163471,8393.700950472834
"POLYGON ((13.346539 52.45, 13.334122 52.450007, 13.328741 52.450185, 13.3292699 52.4540091, 13.330379 52.46159, 13.330492 52.462339, 13.33294 52.47856, 13.334768 52.490646, 13.335995 52.49874, 13.33611 52.49949, 13.336224 52.499887, 13.336342 52.499981, 13.34127 52.5, 13.34259 52.5, 13.3429366 52.4999983, 13.343192 52.499995, 13.343882 52.499945, 13.344301 52.499874, 13.3459997 52.4977467, 13.3461354 52.4975678, 13.35398 52.487156, 13.358316 52.481395, 13.36775 52.468855, 13.367788 52.468794, 13.3683011 52.4671324, 13.3686189 52.4596646, 13.368654 52.458751, 13.368758 52.455146, 13.368865 52.45071, 13.3686217 52.4505333, 13.3683417 52.45035, 13.367675 52.4500233, 13.367405 5

# Convert Kmeans hull results to geojson
ipyleaflet provides an easy way to visualize leaflet maps in jupyter notebooks.

Our hull data contains wkt geometry strings that we will use with a small python library to convert the geometry to GeoJson. Once our data is converted to a proper GeoJson feature collection we can use ipyleaflet to easily load and display that data on a map.

For more information on the GeoJson format visit: http://geojson.org/

In [14]:
from geomet import wkt
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

# Collecting the results will give a array of Rows.
hulls_results = df_hulls.collect()
hulls_geojson = {
    "type": "FeatureCollection",
    "features": []
}
for hull in hulls_results:
    hull = hull.asDict(True)
    output_geojson = {
        "type": "Feature",
        "geometry": {},
        "properties": {}
    }
    # Convert geometry to geojson with geomet
    geom = wkt.loads(hull["geom"])
    output_geojson["geometry"] = geom
    for propKey in hull:
        if propKey != "geom":
            output_geojson["properties"][propKey] = hull[propKey]
    hulls_geojson["features"].append(output_geojson)
print("Count: {0} Features".format(len(hulls_geojson["features"])))

▸,:,


<IPython.core.display.Javascript object>

Count: 8 Features


<IPython.core.display.Javascript object>

In [15]:
center = [52.54, 13.49]
zoom = 10

▸,:,


In [16]:
m = Map(center=center, zoom=zoom)
g = GeoJSON(data=hulls_geojson)
m.add_layer(g)
m

▸,:,


Map(center=[52.54, 13.49], layers=(TileLayer(attribution='Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors', options=['opacity', 'tile_size', 'attribution', 'min_zoom', 'detect_retina', 'max_zoom'], url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'), GeoJSON(data={'type': 'FeatureCollection', 'features': [{'properties': {'Density': 3574.0666345050117, 'ClusterIndex': 6, 'Count': 34677, 'Area': 9.702393252889806}, 'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[13.4439443, 52.4500005], [13.435513, 52.450011], [13.4206216, 52.4501], [13.4124933, 52.451955], [13.412206, 52.452025], [13.4134, 52.4556], [13.414353, 52.458444], [13.416086, 52.460444], [13.41833, 52.462976], [13.421366, 52.466393], [13.4348, 52.4815], [13.4362, 52.4812], [13.4450555, 52.4791942], [13.451483, 52.477729], [13.459015, 52.475988], [13.46339, 52.470942], [13.463752, 52.469908], [13.464018, 52.469147], [13.469976, 52.452055], [13.468309, 52.450024], [13.46598, 52.450009], [13.4646785, 52.4500018], [13.4644143, 52.4500008], [13.4439443, 52.4500005]]]}}, {'properties': {'Density': 4399.1459904984495, 'ClusterIndex': 1, 'Count': 53307, 'Area': 12.117579210859516}, 'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[13.492451, 52.450008], [13.49181, 52.45005], [13.4700016, 52.4520566], [13.4676098, 52.4588682], [13.46515, 52.46591], [13.4637665, 52.4698877], [13.463431, 52.470871], [13.4648923, 52.4813003], [13.469224, 52.489748], [13.470204, 52.491657], [13.473088, 52.497273], [13.4747891, 52.4999943], [13.478891, 52.499998], [13.491197, 52.4999994], [13.496814, 52.499983], [13.496944, 52.4999743], [13.497525, 52.4998866], [13.4982016, 52.4997], [13.4985116, 52.4996116], [13.4987416, 52.4995333], [13.4999883, 52.4990983], [13.499999, 52.498989], [13.499999, 52.462982], [13.499998, 52.452558], [13.499973, 52.451963], [13.499892, 52.450439], [13.499132, 52.450014], [13.497535, 52.450012], [13.492451, 52.450008]]]}}, {'properties': {'Density': 5235.153428916902, 'ClusterIndex': 7, 'Count': 58564, 'Area': 11.186682643629085}, 'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[13.3958883, 52.45001], [13.385129, 52.4500142], [13.378801, 52.45002], [13.3721351, 52.4501419], [13.368953, 52.450731], [13.36895, 52.4507383], [13.368639, 52.460234], [13.368483, 52.467539], [13.36854, 52.46792], [13.36896, 52.468902], [13.36923, 52.46933], [13.3703966, 52.4711074], [13.372795, 52.47476], [13.388901, 52.4992849], [13.389, 52.4994], [13.389111, 52.499513], [13.3915598, 52.4998056], [13.3931416, 52.4995899], [13.3944499, 52.49875], [13.394779, 52.498349], [13.39517, 52.49756], [13.3953339, 52.497216], [13.401532, 52.483614], [13.410421, 52.464095], [13.41233, 52.459899], [13.412791, 52.457626], [13.413, 52.4556], [13.41208, 52.452048], [13.405199, 52.450043], [13.3958883, 52.45001]]]}}, {'properties': {'Density': 5407.7366159924195, 'ClusterIndex': 4, 'Count': 42320, 'Area': 7.825824925505086}, 'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[13.413604, 52.457997], [13.413302, 52.458018], [13.412699, 52.459206], [13.412442, 52.459736], [13.410375, 52.464214], [13.409933, 52.465178], [13.4014989, 52.4836902], [13.401017, 52.484748], [13.399182, 52.488778], [13.395927, 52.495927], [13.395442, 52.496996], [13.3951663, 52.497611], [13.3948768, 52.4982626], [13.3945946, 52.4996212], [13.3949836, 52.4999329], [13.3949967, 52.4999417], [13.39697, 52.4999667], [13.40787, 52.4999899], [13.4156276, 52.4999992], [13.41718, 52.5], [13.43157, 52.5], [13.431861, 52.499982], [13.43195, 52.49995], [13.43205, 52.499428], [13.4328, 52.49458], [13.4340306, 52.4866159], [13.434322, 52.484665], [13.434575, 52.48276], [13.434686, 52.4813836], [13.420205, 52.4650883], [13.416164, 52.460547], [13.414092, 52.458279], [13.413604, 52.457997]]]}}, {'properties': {'Density': 8393.700950472834, 'ClusterIndex': 3, 'Count': 101760, 'Area': 12.123376875163471}, 'type': 'Feature', 'geometry': {'t